<a href="https://colab.research.google.com/github/Teasotea/DialogSystem/blob/main/ConversationalAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
!pip install transformers

In [160]:
!pip install nltk

In [161]:
import numpy as np
import pandas as pd
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
from sklearn.svm import OneClassSVM
import re

# Part I: Classification

## Greeting Classification

In [162]:
training_data_greet, training_data_bye = [], []
greetings =list(dict.fromkeys(['hi', "hola", 'hey', 'hello','morning', 'evening', 'good day', 'good morning', 'greetings', 'howdy', 'welcome', 'bonjour',
             'buenas noches', 'buenos dias', 'salutation', 'salut', 'hail', 'salaam', 'aloha', 'good wishes',
             'aloha', 'yoo-hoo', 'yawp', 'It’s always a pleasure to see you.', 'oh', 'ave', 'yo', 'hi there', 
             "hi", "hello", "hey", "helloo", "hellooo", "g morining", "gmorning", "good morning", "morning", "good day", "good afternoon", "good evening", "greetings", "greeting", 
             "good to see you", "its good seeing you", "g’day", "howdy"]))
goodbyes = list(dict.fromkeys(['bye', 'exit', 'quit', 'I’m off', 
                               'goodbye', 'Goodnight', 'good night !', 'bye-bye',
                               'See ya!', 'Ciao', 'Adios']))

for i in greetings:
  training_data_greet.append({"class":"greeting", "sentence":i})
for i in goodbyes:
  training_data_bye.append({'class':'goodbyes', 'sentence': i})
  
greet_df = pd.DataFrame(training_data_greet)
bye_df = pd.DataFrame(training_data_bye)

bye_df

,class,sentence
0,goodbyes,bye
1,goodbyes,exit
2,goodbyes,quit
3,goodbyes,I’m off
4,goodbyes,goodbye
5,goodbyes,Goodnight
6,goodbyes,good night !
7,goodbyes,bye-bye
8,goodbyes,See ya!
9,goodbyes,Ciao


In [163]:
greetings_test = ['hi, there!','psst, can you tell me, which algorithm would work best if I have imbalanced dataset?','What\'s your favourite music?','Hello? Somebody is here?', 'people often say welcome to guests','How many answers can you provide on this topic?',"hola, hola!", 'tell me about yourself','are you a bot','which movie would ypu suggest to watch this evening?', 'which laptop is the best for Data Science?', 'What about language generation task?','hey, bot:)', 'water']
testy_greet = np.array([ 1, -1, -1, 1, -1, -1, 1, -1,  -1, -1, -1,  -1,  1, -1])

bye_test = ['bye, bot:)', 'OK, then see you tomorrow', 'what does \'exit\' mean?', 'I\'ll take off my hat behind you, bot', 'Let\'s quit this conversation', 'How to translate \'adios\'?', 'Catch me, if you can!', 'Too boring( OK, see you later then']
testy_bye = np.array([ 1, 1, -1, -1, 1, -1, -1, 1])
# array([ 1, -1, -1, -1, -1, -1, -1, -1])

### One-Class Classification with SVM

In [164]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score

In [166]:
num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [167]:
tokenizer_greet = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer_greet.fit_on_texts(greetings)
word_index_greet = tokenizer_greet.word_index
train_sequences_greet = tokenizer_greet.texts_to_sequences(greetings)

# Get max training sequence length
maxlen_greet = 10 #max([len(x) for x in train_sequences_greet])

# Pad the training sequences
train_padded_greet = pad_sequences(train_sequences_greet, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)

# Output the results of our work
print("Word index:\n", word_index_greet)
print("\nTraining sequences:\n", train_sequences_greet)
# print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded_greet.shape)
print("Training sequences data type:", type(train_sequences_greet))
print("Padded Training sequences data type:", type(train_padded_greet))

test_sequences_greet = tokenizer_greet.texts_to_sequences(greetings_test)
test_padded_greet = pad_sequences(test_sequences_greet, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)

print("Testing sequences:\n", test_sequences_greet)
print("\nPadded testing sequences:\n", test_padded_greet)
print("\nPadded testing shape:",test_padded_greet.shape)

for x, y in zip(greetings_test, test_padded_greet):
  print('{} -> {}'.format(x, y))

Word index:
 {'<UNK>': 1, 'good': 2, 'you': 3, 'hi': 4, 'morning': 5, 'evening': 6, 'to': 7, 'see': 8, 'hola': 9, 'hey': 10, 'hello': 11, 'day': 12, 'greetings': 13, 'howdy': 14, 'welcome': 15, 'bonjour': 16, 'buenas': 17, 'noches': 18, 'buenos': 19, 'dias': 20, 'salutation': 21, 'salut': 22, 'hail': 23, 'salaam': 24, 'aloha': 25, 'wishes': 26, 'yoo': 27, 'hoo': 28, 'yawp': 29, 'it’s': 30, 'always': 31, 'a': 32, 'pleasure': 33, 'oh': 34, 'ave': 35, 'yo': 36, 'there': 37, 'helloo': 38, 'hellooo': 39, 'g': 40, 'morining': 41, 'gmorning': 42, 'afternoon': 43, 'greeting': 44, 'its': 45, 'seeing': 46, 'g’day': 47}

Training sequences:
 [[4], [9], [10], [11], [5], [6], [2, 12], [2, 5], [13], [14], [15], [16], [17, 18], [19, 20], [21], [22], [23], [24], [25], [2, 26], [27, 28], [29], [30, 31, 32, 33, 7, 8, 3], [34], [35], [36], [4, 37], [38], [39], [40, 41], [42], [2, 43], [2, 6], [44], [2, 7, 8, 3], [45, 2, 46, 3], [47]]

Padded training shape: (37, 10)
Training sequences data type: <class '

In [168]:
tokenizer_bye= Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer_bye.fit_on_texts(goodbyes)
word_index_bye = tokenizer_bye.word_index
train_sequences_bye = tokenizer_bye.texts_to_sequences(goodbyes)

# Get max training sequence length
maxlen_bye = 10 #max([len(x) for x in train_sequences_bye])

# Pad the training sequences
train_padded_bye = pad_sequences(train_sequences_bye, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)

# Output the results of our work
print("Word index:\n", word_index_bye)
print("\nTraining sequences:\n", train_sequences_bye)
# print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded_bye.shape)
print("Training sequences data type:", type(train_sequences_bye))
print("Padded Training sequences data type:", type(train_padded_bye))

test_sequences_bye = tokenizer_bye.texts_to_sequences(bye_test)
test_padded_bye = pad_sequences(test_sequences_bye, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)

print("Testing sequences:\n", test_sequences_bye)
print("\nPadded testing sequences:\n", test_padded_bye)
print("\nPadded testing shape:",test_padded_bye.shape)

for x, y in zip(bye_test, test_padded_bye):
  print('{} -> {}'.format(x, y))

Word index:
 {'<UNK>': 1, 'bye': 2, 'exit': 3, 'quit': 4, 'i’m': 5, 'off': 6, 'goodbye': 7, 'goodnight': 8, 'good': 9, 'night': 10, 'see': 11, 'ya': 12, 'ciao': 13, 'adios': 14}

Training sequences:
 [[2], [3], [4], [5, 6], [7], [8], [9, 10], [2, 2], [11, 12], [13], [14]]

Padded training shape: (11, 10)
Training sequences data type: <class 'list'>
Padded Training sequences data type: <class 'numpy.ndarray'>
Testing sequences:
 [[2, 1], [1, 1, 11, 1, 1], [1, 1, 1, 1], [1, 1, 6, 1, 1, 1, 1, 1], [1, 4, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 11, 1, 1, 1]]

Padded testing sequences:
 [[ 2  1  0  0  0  0  0  0  0  0]
 [ 1  1 11  1  1  0  0  0  0  0]
 [ 1  1  1  1  0  0  0  0  0  0]
 [ 1  1  6  1  1  1  1  1  0  0]
 [ 1  4  1  1  0  0  0  0  0  0]
 [ 1  1  1  1  0  0  0  0  0  0]
 [ 1  1  1  1  1  0  0  0  0  0]
 [ 1  1  1 11  1  1  1  0  0  0]]

Padded testing shape: (8, 10)
bye, bot:) -> [2 1 0 0 0 0 0 0 0 0]
OK, then see you tomorrow -> [ 1  1 11  1  1  0  0  0  0  0]
what does '

### Greetings

In [169]:
model = OneClassSVM(gamma='scale', nu=0.01)
model.fit(train_padded_greet)

OneClassSVM(nu=0.01)

In [170]:
#+1 for normal examples, so-called inliers, and a -1 for outliers.
yhat_greet = model.predict(test_padded_greet)

In [171]:
yhat_greet

array([ 1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1])

In [172]:
score = f1_score(testy_greet, yhat_greet, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 1.000


In [173]:
def classify_greet(sent):
  l = [sent]
  test_sequences = tokenizer_greet.texts_to_sequences(l)
  test_padded= pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)
  pred_class_greet = model.predict(test_padded)
  cl = int(*pred_class_greet)
  return  cl
  
print(classify_greet('Hey! Lets talk, bot'))

1


### Gooodbyes

In [174]:
model_bye = OneClassSVM(gamma='scale', nu=0.01)
model_bye.fit(train_padded_bye)

OneClassSVM(nu=0.01)

In [175]:
yhat_bye = model_bye.predict(test_padded_bye)
yhat_bye

array([ 1, -1, -1, -1, -1, -1, -1, -1])

In [176]:
score = f1_score(testy_bye, yhat_bye, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 0.727


*I need to gather bigger dataset (or use better algirothm) in order to improve the model* 



In [177]:
def classify_goodbyes(sent):
  l = [sent]
  test_sequences = tokenizer_bye.texts_to_sequences(l)
  test_padded= pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)
  pred_class = model_bye.predict(test_padded)
  cl = int(*pred_class)
  return  cl
  
print(classify_goodbyes('Bye, bot'))

1


# Part II: Question Answering

In [178]:
!pip install datasets

In [179]:
import datasets
import requests

[ds for ds in datasets.list_datasets() if 'science' in ds.lower()]

['multi_x_science_sum',
 'web_of_science',
 'bigscience/LanguageResourceCatalogue',
 'bigscience/P3',
 'bigscience/mc4-sampled',
 'bigscience/open_subtitles_monolingual',
 'bigscience-catalogue-data-dev/lm_code_github-eval_subset',
 'bigscience-catalogue-lm-data/lm_ar_wikibooks',
 'bigscience-catalogue-lm-data/lm_ar_wikibooks_filtered',
 'bigscience-catalogue-lm-data/lm_ar_wikinews_filtered',
 'bigscience-catalogue-lm-data/lm_ar_wikiquote_filtered',
 'bigscience-catalogue-lm-data/lm_ar_wikisource',
 'bigscience-catalogue-lm-data/lm_ar_wikisource_filtered',
 'bigscience-catalogue-lm-data/lm_ar_wikiversity_filtered',
 'bigscience-catalogue-lm-data/lm_ar_wiktionary_filtered',
 'bigscience-catalogue-lm-data/lm_ca_wikibooks_filtered',
 'bigscience-catalogue-lm-data/lm_ca_wikimedia_filtered',
 'bigscience-catalogue-lm-data/lm_ca_wikinews_filtered',
 'bigscience-catalogue-lm-data/lm_ca_wikiquote_filtered',
 'bigscience-catalogue-lm-data/lm_ca_wikisource_filtered',
 'bigscience-catalogue-lm-da

In [180]:
qa_ds = datasets.load_dataset('squad', streaming = False)
# qa_ds = datasets.load_dataset('web_of_science','WOS5736')
qa_ds

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [181]:
qa_ds['train'].description

'Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.\n'

In [182]:
print(qa_ds['train'].dataset_size)
qa_ds['train'].features

89846964


{'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}

In [183]:
qa_ds['train'].to_pandas().head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


## Data Science Interview Dataset

In [184]:
qa_data = pd.read_csv('https://raw.githubusercontent.com/Kizuna-Cheng/Data_Science_Interviews_NLP/main/data.csv')
qa_data

,Category,Questions,Answers
0,Linear Regression,What are the assumptions required for linear r...,There are four assumptions associated with a l...
1,Statistics,What is collinearity? What is multicollinearit...,Collinearity is a linear association between t...
2,Linear Regression,What are the drawbacks of a linear model?\n,There are a couple of drawbacks of a linear mo...
3,Linear Regression,What are ridge and lasso regression and what a...,Both L1 and L2 regularization are methods used...
4,KNN,How does K-Nearest Neighbor work?,K-Nearest Neighbors is a classification techni...
...,...,...,...
319,Neural Network,What are advantages and disadvantages of using...,Advantages:\n\nStore data on the entire networ...
320,Decision Trees,What is the default method for splitting in de...,"The default method is the Gini Index, which is..."
321,"XGBoost, SVM",Why does XGBoost perform better than SVM?,XGBoos is an ensemble method that uses many tr...
322,Neural Network,"For NLP, what’s the main purpose of using an e...",We use the encoder-decoder model to generate a...


In [185]:
qa_data.Questions[:10].tolist()

['What are the assumptions required for linear regression? What if some of these assumptions are violated?',
 'What is collinearity? What is multicollinearity? How do you deal with it?\n',
 'What are the drawbacks of a linear model?\n',
 'What are ridge and lasso regression and what are the differences between them?',
 'How does K-Nearest Neighbor work?',
 'How can you select k for k means?',
 'Why is Naive Bayes “naive”?',
 'What are the support vectors in SVM?',
 'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?']

In [186]:
qa_data.Answers[:10].tolist()

['There are four assumptions associated with a linear regression model:\nLinearity: The relationship between X and the mean of Y is linear.\nHomoscedasticity: The variance of the residual is the same for any value of X.\nIndependence: Observations are independent of each other.\nNormality: For any fixed value of X, Y is normally distributed.\nExtreme violations of these assumptions will make the results redundant. Small violations of these assumptions will result in a greater bias or variance of the estimate.',
 'Collinearity is a linear association between two predictors. Multicollinearity is a situation where two or more predictors are highly linearly related.\nThis can be problematic because it undermines the statistical significance of an independent variable. While it may not necessarily have a large impact on the model’s accuracy, it affects the variance of the prediction and reduces the quality of the interpretation of the independent variables.\nYou could use the Variance Infla

In [187]:
test_data = [
             'What does linear regression stands for?',
 'What is the differencebetween collinearity and multicollinearity?',
 'What are the cons of using a linear model?\n',
 'What are ridge and lasso regression?',
 'How does K-Nearest neighbor work?',
 'How to select k for k means?',
 'Why is Naive Bayes “naive”?',
 'When should I use SVM?',
'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?']

## QA Baseline

In [188]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])

In [189]:
def getNaiveAnswer(q):
    row = qa_data.loc[qa_data['Questions'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answers"].values[0], 1, row["Questions"].values[0]
    else: return "Sorry, I didn't get you", 0, ""
print(getNaiveAnswer('How does K-Nearest Neighbor work?'))
getResults(test_data, getNaiveAnswer)

("Sorry, I didn't get you", 0, '')


,Q,Prediction,A,Score
0,What does linear regression stands for?,,"Sorry, I didn't get you",0
1,What is the differencebetween collinearity and...,,"Sorry, I didn't get you",0
2,What are the cons of using a linear model?\n,,"Sorry, I didn't get you",0
3,What are ridge and lasso regression?,What are ridge and lasso regression and what a...,Both L1 and L2 regularization are methods used...,1
4,How does K-Nearest neighbor work?,,"Sorry, I didn't get you",0
5,How to select k for k means?,,"Sorry, I didn't get you",0
6,Why is Naive Bayes “naive”?,,"Sorry, I didn't get you",0
7,When should I use SVM?,,"Sorry, I didn't get you",0
8,What is pruning in decision trees?,What is pruning in decision trees?,Pruning is a technique in machine learning and...,1
9,What are random forests? Why is Naive Bayes be...,,"Sorry, I didn't get you",0


## Levenstein distance

In [190]:
pip install python-Levenshtein

In [191]:
from Levenshtein import ratio

In [266]:
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in qa_data.iterrows():
        score = ratio(row["Questions"], q)
        if score >= 0.9:  # I'm sure, stop here 
            return  row["Answers"], score, row["Questions"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answers"]
            prediction = row["Questions"]

    return answer, max_score, prediction

In [267]:
getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,What does linear regression stands for?,What methods for solving linear regression do ...,"To solve linear regression, you need to find t...",0.659574
1,What is the differencebetween collinearity and...,What is the difference between online and batc...,"Batch learning, also known as offline learning...",0.786885
2,What are the cons of using a linear model?\n,What are the drawbacks of a linear model?\n,There are a couple of drawbacks of a linear mo...,0.823529
3,What are ridge and lasso regression?,What are the main assumptions of linear regres...,There are several assumptions of linear regres...,0.689655
4,How does K-Nearest neighbor work?,How does K-Nearest Neighbor work?,K-Nearest Neighbors is a classification techni...,0.969697
5,How to select k for k means?,How to select K for K-means?,"Domain knowledge, i.e. an expert knows the val...",0.892857
6,Why is Naive Bayes “naive”?,Why is Naive Bayes “naive”?,Naive Bayes is naive because it holds a strong...,1.000000
7,When should I use SVM?,When would you use Adam and when SGD?,"Adam tends to converge faster, while SGD often...",0.576271
8,What is pruning in decision trees?,What is pruning in decision trees?,Pruning is a technique in machine learning and...,1.000000
9,What are random forests? Why is Naive Bayes be...,What are random forests? Why is Naive Bayes be...,Random forests are an ensemble learning techni...,1.000000


## BERT info retriever

In [197]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizerQABert = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
modelQABert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [ ]:
question, text = "What is collinearity?", qa_data.Answers[1]
print(qa_data.Answers[1])

In [199]:
def answer_with_BERT(question, text):
  inputs = tokenizerQABert(question, text, return_tensors="pt")
  with torch.no_grad():
    outputs = modelQABert(**inputs)
  answer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  return tokenizerQABert.decode(predict_answer_tokens)

In [234]:
#[CLS] - no answer
print(answer_with_BERT("What is collinearity?", qa_data.Answers[1]))

a linear association between two predictors
[CLS]


In [262]:
def find_answer(similiar_question):
  if getApproximateAnswer(similiar_question)[1]<0.6:
    return "Sorry, I didn't get you"
  q = getApproximateAnswer(similiar_question)[2]
  answ = qa_data[qa_data['Questions']==q].iloc[0]['Answers']
  short_answer = answer_with_BERT(similiar_question, answ)
  if short_answer != '[CLS]':
    return short_answer
  else: return "Sorry, I didn't get you"

In [251]:
print(getApproximateAnswer("What is log regression?"))
print(find_answer('What is log regression?'))

('Logistic regression is also known as the logit model. It is a technique used to forecast the binary outcome from a linear combination of predictor variables.', 0.9019607843137255, 'What is logistic regression?')
a technique used to forecast the binary outcome from a linear combination of predictor variables


In [263]:
print(getApproximateAnswer("What is collinearity?"))
print(find_answer('What is collinearity?'))

('Overfitting is an error where the model ‘fits’ the data too well, resulting in a model with high variance and low bias. As a consequence, an overfit model will inaccurately predict new data points even though it has a high accuracy on the training data.', 0.6829268292682927, 'What is overfitting?')
Sorry, I didn't get you


In [265]:
print(getApproximateAnswer("When should I use SVM?"))
print(find_answer('When should I use SVM?'))

("Sorry, I didn't get you.", 0.576271186440678, 'When would you use Adam and when SGD?')
Sorry, I didn't get you


In [253]:
find_answer('What is linear regression?')

'a technique used to forecast the binary outcome from a linear combination of predictor variables'

# Part III: Natural Language Generation

In [194]:
# checkpoint 
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
modelNLG = AutoModelForCausalLM.from_pretrained(checkpoint)

# Part IV: Bot Development

In [259]:
class ChatBot():
    def __init__(self):
        self.chat_history_ids = None
        self.bot_input_ids = None
        self.end_chat = False
        self.is_greeting = False
        self.is_question = False
        self.text_input = None
        self.welcome()
        
    def welcome(self):
        print("Initializing ChatBot ...")
        time.sleep(2)
        print('Type any word to start the conversation!\n')
        time.sleep(3)

        
    def user_input(self):
        text = str(input("User    >> "))
        if classify_goodbyes(text.lower()) == 1:
            self.end_chat=True
            print('ChatBot >>  See you soon! Bye!')
            time.sleep(1)
            print('\nQuitting ChatBot ...')
        else:
            if classify_greet(text) == -1:
              self.text_input = text
              if find_answer(self.text_input) != 'Sorry, I didn\'t get you':
                self.is_question = True
            else: 
              self.is_greeting = True

    def bot_greet(self):
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)
        self.is_greeting = False

    def bot_response(self):
        self.new_user_input_ids = tokenizer2.encode(self.text_input + tokenizer2.eos_token, \
                                                       return_tensors='pt')
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
        else:
            self.bot_input_ids = self.new_user_input_ids
         
        self.chat_history_ids = modelNLG.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer2.eos_token_id)
            
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        if response == "":
            response = self.random_response()
        print('ChatBot >>  '+ response)
    
    def bot_answer(self):
      print('ChatBot >>  '+ find_answer(self.text_input))
      self.is_question = False
        
    def random_response(self):
        i = -1
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        while response == '':
            i = i-1
            response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
        return reply

In [261]:
bot = ChatBot()
while True:
    bot.user_input()
    if bot.end_chat:
        break
    if bot.is_greeting == False:
      if bot.is_question == True:
        bot.bot_answer()
      else:
        bot.bot_response() 
    else: bot.bot_greet()  

Initializing ChatBot ...
Type any word to start the conversation!

User    >> When should I use SVM?
ChatBot >>  When you have a good enough team.
User    >> I get you
ChatBot >>  I get you too
User    >> bye
ChatBot >>  See you soon! Bye!

Quitting ChatBot ...
